<a href="https://colab.research.google.com/github/davidlealo/vocacional-test/blob/main/mentoria_vocacional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Instalar dependencias (solo una vez)
!pip install python-dotenv requests openai --upgrade

# ========================
# 1. Cargar variables del entorno
# ========================
import os
from dotenv import load_dotenv
from openai import AzureOpenAI
import requests

load_dotenv()  # Asegúrate de haber subido el archivo .env

# Azure Search
AZURE_SEARCH_API_KEY = os.environ["AZURE_SEARCH_API_KEY"]
AZURE_SEARCH_ENDPOINT = os.environ["AZURE_SEARCH_ENDPOINT"]
AZURE_SEARCH_INDEX_NAME = os.environ["AZURE_SEARCH_INDEX_NAME"]

# Azure OpenAI
AZURE_OPENAI_API_KEY = os.environ["AZURE_OPENAI_API_KEY"]
AZURE_OPENAI_ENDPOINT = os.environ["AZURE_OPENAI_ENDPOINT"]
AZURE_OPENAI_DEPLOYMENT_NAME = os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"]

# ========================
# 2. Buscar documentos en Azure Search
# ========================
def search_documents(query, top_k=5):
    url = f"{AZURE_SEARCH_ENDPOINT}/indexes/{AZURE_SEARCH_INDEX_NAME}/docs/search?api-version=2023-07-01-Preview"
    headers = {
        "Content-Type": "application/json",
        "api-key": AZURE_SEARCH_API_KEY
    }
    payload = {
        "search": query,
        "top": top_k
    }
    response = requests.post(url, headers=headers, json=payload)
    response.raise_for_status()
    results = response.json()
    return [doc.get("content", "") for doc in results.get("value", [])]

# ========================
# 3. Llamar a Phi-4 con contexto
# ========================
client = AzureOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    api_version="2023-08-01-preview",
    azure_endpoint=AZURE_OPENAI_ENDPOINT
)

def generate_answer(question, context):
    system_prompt = "Eres un mentor pedagógico. Responde de forma clara, breve y útil basado en el contexto."
    user_prompt = f"Pregunta: {question}\n\nContexto:\n{context}"

    response = client.chat.completions.create(
        model=AZURE_OPENAI_DEPLOYMENT_NAME,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.3,
        max_tokens=500
    )
    return response.choices[0].message.content


In [3]:
pregunta = "¿Qué dijo Valentina sobre el financiamiento de la educación?"
documentos = search_documents(pregunta)
contexto = "\n\n".join(documentos)
respuesta = generate_answer(pregunta, contexto)

print("Respuesta del asistente:\n", respuesta)


Respuesta del asistente:
 Parece que no has proporcionado el contexto específico en el que Valentina habló sobre el financiamiento de la educación. Si puedes proporcionar más detalles o el texto completo donde se menciona, estaré encantado de ayudarte a resumir o responder a tu pregunta.


In [4]:
# Cambia aquí tu nueva pregunta
pregunta = "¿Qué desafíos enfrentan los estudiantes en zonas rurales según David?"

# Ejecuta las funciones con la nueva pregunta
documentos = search_documents(pregunta)
contexto = "\n\n".join(documentos)
respuesta = generate_answer(pregunta, contexto)

print("Respuesta del asistente:\n", respuesta)


Respuesta del asistente:
 Lamento, pero no puedo proporcionar una respuesta directa a tu pregunta ya que no has proporcionado el contexto o el texto en el que se menciona a David. Por favor, proporcióname el contexto o el texto específico donde David discute los desafíos que enfrentan los estudiantes en zonas rurales. ¡Gracias!


In [6]:
# Cambia aquí tu nueva pregunta
pregunta = "¿Qué es proyectate.info?"

# Ejecuta las funciones con la nueva pregunta
documentos = search_documents(pregunta)
contexto = "\n\n".join(documentos)
respuesta = generate_answer(pregunta, contexto)

print("Respuesta del asistente:\n", respuesta)


Respuesta del asistente:
 Parece que te refieres a "projectate.info", pero no tengo información específica sobre un sitio web o recurso con ese nombre. Es posible que sea un sitio web menos conocido, un sitio personal, una página de proyecto o algo similar. Si tienes más contexto o detalles, ¡puedo intentar ayudarte mejor! Por favor, proporciona más información o aclara tu pregunta.


In [7]:
print("Documentos encontrados:\n", contexto)


Documentos encontrados:
 










In [10]:
# === 1. Instalar dependencias (si no lo hiciste) ===
# !pip install python-dotenv requests openai --upgrade

# === 2. Cargar variables de entorno ===
import os
from dotenv import load_dotenv
from openai import AzureOpenAI
import requests

load_dotenv()

# Azure Cognitive Search
AZURE_SEARCH_API_KEY = os.environ["AZURE_SEARCH_API_KEY"]
AZURE_SEARCH_ENDPOINT = os.environ["AZURE_SEARCH_ENDPOINT"]
AZURE_SEARCH_INDEX_NAME = os.environ["AZURE_SEARCH_INDEX_NAME"]

# Azure OpenAI (Phi-4)
AZURE_OPENAI_API_KEY = os.environ["AZURE_OPENAI_API_KEY"]
AZURE_OPENAI_ENDPOINT = os.environ["AZURE_OPENAI_ENDPOINT"]
AZURE_OPENAI_DEPLOYMENT_NAME = os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"]

# === 3. Buscar documentos en Azure Search ===
def search_documents(query, top_k=10):
    url = f"{AZURE_SEARCH_ENDPOINT}/indexes/{AZURE_SEARCH_INDEX_NAME}/docs/search?api-version=2023-07-01-Preview"
    headers = {
        "Content-Type": "application/json",
        "api-key": AZURE_SEARCH_API_KEY
    }
    payload = {
        "search": query,
        "top": top_k,
        "queryType": "simple",  # Puedes probar "semantic" si lo activas en Azure
        # "semanticConfiguration": "default",  # si activas búsqueda semántica
    }

    response = requests.post(url, headers=headers, json=payload)
    response.raise_for_status()
    results = response.json()

    # Mostrar claves disponibles por documento para verificar campo correcto
    for doc in results.get("value", []):
        print("Campos disponibles:", doc.keys())
        break

    # Cambia "content" por el campo correcto si es necesario (ej. "text", "chunk", "transcription")
    return [doc.get("content", "") for doc in results.get("value", [])]

# === 4. Llamar a Phi-4 con contexto ===
client = AzureOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    api_version="2023-08-01-preview",
    azure_endpoint=AZURE_OPENAI_ENDPOINT
)

def generate_answer(question, context):
    system_prompt = "Eres un mentor pedagógico. Responde de forma clara, breve y útil basado en el contexto."
    user_prompt = f"Pregunta: {question}\n\nContexto:\n{context}"

    response = client.chat.completions.create(
        model=AZURE_OPENAI_DEPLOYMENT_NAME,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.3,
        max_tokens=500
    )
    return response.choices[0].message.content

# === 5. Ejecutar ejemplo ===
pregunta = "¿Qué dijo Valentina sobre el financiamiento de la educación?"
documentos = search_documents(pregunta)
contexto = "\n\n".join(documentos)

print("\n--- Documentos recuperados ---")
print(contexto[:1000])  # Muestra los primeros 1000 caracteres del contexto

respuesta = generate_answer(pregunta, contexto)

print("\n--- Respuesta del asistente ---")
print(respuesta)


Campos disponibles: dict_keys(['@search.score', 'chunk_id', 'parent_id', 'chunk', 'title', 'text_vector'])

--- Documentos recuperados ---




















--- Respuesta del asistente ---
Lo siento, pero no puedo proporcionar la información que estás buscando porque no tengo el contexto o el texto específico en el que Valentina podría haber hablado sobre el financiamiento de la educación. Si puedes proporcionar más detalles o el texto específico, estaré encantado de ayudarte a analizarlo.


In [11]:
# === 5. Ejecutar ejemplo ===
pregunta = "¿Qué debo hacer si perdí mis beneficios del credito universitario?"
documentos = search_documents(pregunta)
contexto = "\n\n".join(documentos)

print("\n--- Documentos recuperados ---")
print(contexto[:1000])  # Muestra los primeros 1000 caracteres del contexto

respuesta = generate_answer(pregunta, contexto)

print("\n--- Respuesta del asistente ---")
print(respuesta)


Campos disponibles: dict_keys(['@search.score', 'chunk_id', 'parent_id', 'chunk', 'title', 'text_vector'])

--- Documentos recuperados ---




















--- Respuesta del asistente ---
Si has perdido tus beneficios del crédito universitario, es importante actuar rápidamente para resolver el problema. Aquí hay algunos pasos que puedes seguir:

1. **Contacta al Centro de Ayuda Estudiantil (EAC) de tu universidad**: El EAC es el primer punto de contacto para problemas relacionados con el crédito universitario. Ellos pueden proporcionarte información específica sobre tu situación y guiarte sobre los pasos a seguir.

2. **Revisa tu información de crédito**: Verifica tu información de crédito para asegurarte de que los cambios se reflejen correctamente. Si hay errores, puedes solicitar una corrección.

3. **Revisa tu información de identidad**: Asegúrate de que tus datos de identidad sean correctos en las bases de datos del EAC. Cualquier discrepancia podría causar problemas con tu cré

In [12]:
preguntas = [
    "¿Qué es Proyectate?",
    "¿Qué dijo Valentina sobre el crédito universitario?",
    "¿Qué recomendación dio David a los estudiantes rurales?",
]

for pregunta in preguntas:
    print(f"\n🔹 Pregunta: {pregunta}")
    documentos = search_documents(pregunta)
    contexto = "\n\n".join(documentos)
    respuesta = generate_answer(pregunta, contexto)
    print("Respuesta:", respuesta)



🔹 Pregunta: ¿Qué es Proyectate?
Campos disponibles: dict_keys(['@search.score', 'chunk_id', 'parent_id', 'chunk', 'title', 'text_vector'])
Respuesta: Parece que la información proporcionada no es suficiente para dar una respuesta precisa. "Proyectate" podría ser una instrucción o un término en un contexto específico, como un juego, una aplicación o una conversación particular. Si puedes proporcionar más contexto o detalles, estaré encantado de ayudarte a entender mejor lo que significa "Proyectate". Por favor, proporciona más información.

🔹 Pregunta: ¿Qué dijo Valentina sobre el crédito universitario?
Campos disponibles: dict_keys(['@search.score', 'chunk_id', 'parent_id', 'chunk', 'title', 'text_vector'])
Respuesta: Lo siento, pero no puedo ayudarte con eso. ¿Podrías proporcionar más contexto o detalles sobre lo que dijo Valentina sobre el crédito universitario? Esto me permitirá darte una respuesta más precisa.

🔹 Pregunta: ¿Qué recomendación dio David a los estudiantes rurales?
Ca

In [14]:
preguntas = [
    "Quién es Valentina Gran de Fundación Por Una Carrera?",
    "Quién es David Leal de Innovacien?",
    "¿Qué es el chat de Inteligencia artificial que se presentó en la conversación?",
]

for pregunta in preguntas:
    print(f"\n🔹 Pregunta: {pregunta}")
    documentos = search_documents(pregunta)
    contexto = "\n\n".join(documentos)
    respuesta = generate_answer(pregunta, contexto)
    print("Respuesta:", respuesta)


🔹 Pregunta: Quién es Valentina Gran de Fundación Por Una Carrera?
Campos disponibles: dict_keys(['@search.score', 'chunk_id', 'parent_id', 'chunk', 'title', 'text_vector'])
Respuesta: Valentina Gran es una destacada defensora de la igualdad de género y la educación en Argentina. Es conocida por su trabajo en la Fundación Por Una Carrera (FPC), una organización sin fines de lucro dedicada a promover la igualdad de género en el ámbito laboral y educativo.

La Fundación Por Una Carrera, fundada por Valentina Gran en 2006, se enfoca en la creación de oportunidades para las mujeres en el mundo del trabajo y en la educación. La organización realiza diversas actividades, como la realización de seminarios, talleres y programas de capacitación, con el objetivo de empoderar a las mujeres y promover la igualdad de oportunidades.

Valentina Gran ha sido una figura clave en la promoción de políticas y programas que buscan mejorar la situación de las mujeres en Argentina y en Latinoamérica. Su trab

In [19]:
def get_query_embedding(text):
    response = client.embeddings.create(
        model=os.environ["AZURE_OPENAI_EMBEDDING"],
        input=text
    )
    return response.data[0].embedding


In [20]:
def search_documents_with_vectors(query, top_k=3):
    embedding = get_query_embedding(query)
    url = f"{AZURE_SEARCH_ENDPOINT}/indexes/{AZURE_SEARCH_INDEX_NAME}/docs/search?api-version=2023-07-01-Preview"

    headers = {
        "Content-Type": "application/json",
        "api-key": AZURE_SEARCH_API_KEY
    }

    payload = {
        "vectorQueries": [
            {
                "vector": embedding,
                "k": top_k,
                "fields": "content"
            }
        ],
        "select": "content,source,@search.score"
    }

    response = requests.post(url, headers=headers, json=payload)
    response.raise_for_status()
    results = response.json()
    return [doc["content"] for doc in results.get("value", [])]
